In [1]:
import triton
import triton.language as tl
import torch

# im2col

In [2]:
def _out_hw(H, W, KH, KW, SH, SW, PH, PW, DH, DW):
    H_OUT = (H + 2*PH - DH*(KH-1) - 1)//SH + 1
    W_OUT = (W + 2*PW - DW*(KW-1) - 1)//SW + 1
    return H_OUT, W_OUT

In [3]:
import triton
import triton.language as tl

@triton.jit
def _im2col_kernel_2d(
    x_ptr, cols_ptr,
    N, C, H, W,
    KH, KW, SH, SW, PH, PW, DH, DW,
    H_OUT, W_OUT, K_TOTAL,
    sxn, sxc, sxh, sxw,
    scn, sck, scl,
    BLOCK_X: tl.constexpr,
):
    pid = tl.program_id(0)
    L = H_OUT * W_OUT

    # [n, l] из одномерной сетки
    n = pid // L
    l = pid %  L

    # oh/ow окна и его базовые координаты во входе
    oh = l // W_OUT
    ow = l %  W_OUT
    ih0 = oh * SH - PH
    iw0 = ow * SW - PW

    # Общие константы по этому pid
    base_xn = n * sxn
    base_cols_nl = n * scn + l * scl

    # Основной цикл по K с шагом BLOCK_X
    k0 = 0
    while k0 < K_TOTAL:
        k = k0 + tl.arange(0, BLOCK_X)                      # [BLOCK_X]
        k_mask = k < K_TOTAL

        # Разложение k -> (c, kh, kw)
        r  = k %  (KH * KW)
        c  = k // (KH * KW)
        kh = r // KW
        kw = r %  KW

        # Абсолютные координаты пикселя окна во входе
        ih = ih0 + kh * DH
        iw = iw0 + kw * DW

        # Маска "внутри" входного тензора
        inb = (ih >= 0) & (ih < H) & (iw >= 0) & (iw < W) & k_mask

        # Адреса: x[n, c, ih, iw]
        x_off = base_xn + c * sxc + ih * sxh + iw * sxw

        # Маскированная загрузка (без tl.where — он лишний)
        vals = tl.load(x_ptr + x_off, mask=inb, other=0)

        # Адреса для cols[n, k, l]
        cols_off = base_cols_nl + k * sck

        # Запись с маской
        tl.store(cols_ptr + cols_off, vals, mask=k_mask)

        k0 += BLOCK_X


In [4]:
def im2col_triton(x, kernel_size, stride=1, padding=0, dilation=1, block_k=256):
    assert x.is_cuda
    if isinstance(kernel_size, int): kernel_size = (kernel_size, kernel_size)
    if isinstance(stride, int): stride = (stride, stride)
    if isinstance(padding, int): padding = (padding, padding)
    if isinstance(dilation, int): dilation = (dilation, dilation)

    N, C, H, W = x.shape
    KH, KW = kernel_size
    SH, SW = stride
    PH, PW = padding
    DH, DW = dilation
    # Выходные  H / W  размеры тензора
    H_OUT, W_OUT = _out_hw(H, W, KH, KW, SH, SW, PH, PW, DH, DW)
    # длина выхожного массива
    L = H_OUT * W_OUT
    #  длинны кернела 
    K_TOTAL = C * KH * KW
    # пустышка выходного тензора
    cols = torch.empty((N, K_TOTAL, L), device=x.device, dtype=torch.float32)
    # страйд по входному массиву
    sxn, sxc, sxh, sxw = x.stride()
    # страйд по выходному массиву
    scn, sck, scl = cols.stride()
    # одномерныя сетка
    grid = (N * L,)
    _im2col_kernel_2d[grid](
        x, cols,
        N, C, H, W,
        KH, KW, SH, SW, PH, PW, DH, DW,
        H_OUT, W_OUT, K_TOTAL,
        sxn, sxc, sxh, sxw,
        scn, sck, scl,
        BLOCK_X=block_k,
        num_warps=4, num_stages=2,
    )
    return cols, (H_OUT, W_OUT)

In [5]:
import torch
import torch.nn.functional as F


# батч / каналы/ ширина аarray / высота аarray
N, C, H, W = 1, 3, 2000, 2000
# H / W свёртки
KH, KW = 3, 3
# страйд свертки
SH, SW = 1, 1
#  паддинг свертки
PH, PW = 1, 1
# дилатация свертки
DH, DW = 1, 1
#  размер блока НПУ
block_k = 256

# входной тензор N, C, H, W
x = torch.arange(N*C*H*W, dtype=torch.float32, device='cuda').reshape(N, C, H, W)

# базовый unfold
cols_ref = F.unfold(
    x, kernel_size=(KH, KW),
    dilation=(DH, DW),
    padding=(PH, PW),
    stride=(SH, SW)
)

# im2col_triton
cols_tr, (H_out, W_out) = im2col_triton(
    x,
    kernel_size=(KH, KW),
    stride=(SH, SW),
    padding=(PH, PW),
    dilation=(DH, DW),
    block_k=block_k
)

# # проверим совпадение
# print("cols_ref shape:", cols_ref.shape)
# print("cols_tr   shape:", cols_tr.shape)

# max_abs = (cols_tr - cols_ref).abs().max().item()
# print("Макс. разница:", max_abs)
# print("H_out, W_out =", H_out, W_out)

# # убедимся визуально
# print("\nПервый столбец эталон:")
# print(cols_ref[0, :, 0])
# print("\nПервый столбец Triton:")
# print(cols_tr[0, :, 0])

In [6]:
print("Форма cols_ref:", cols_ref.shape)
print("Один патч 3x3 ", cols_ref[0, :, 0])
print("Первый элемент всех патчей: ",cols_ref[0, 0, :])

Форма cols_ref: torch.Size([1, 27, 4000000])
Один патч 3x3  tensor([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
        0.0000e+00, 2.0000e+03, 2.0010e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 4.0000e+06, 4.0000e+06, 0.0000e+00, 4.0020e+06, 4.0020e+06,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 8.0000e+06, 8.0000e+06,
        0.0000e+00, 8.0020e+06, 8.0020e+06], device='cuda:0')
Первый элемент всех патчей:  tensor([      0.,       0.,       0.,  ..., 3997996., 3997997., 3997998.],
       device='cuda:0')


In [9]:
# %%time
# # входной тензор N, C, H, W
# x = torch.arange(N*C*H*W, dtype=torch.float32, device='cuda').reshape(N, C, H, W)
# for _ in range(1000):
#     cols_tr, (H_out, W_out) = im2col_triton(
#     x,
#     kernel_size=(KH, KW),
#     stride=(SH, SW),
#     padding=(PH, PW),
#     dilation=(DH, DW),
#     block_k=block_k
# )

In [10]:
# %%time
# # входной тензор N, C, H, W
# x = torch.arange(N*C*H*W, dtype=torch.float32, device='cuda').reshape(N, C, H, W)
# for _ in range(1000):
#     # базовый unfold
#     cols_ref = F.unfold(
#         x, kernel_size=(KH, KW),
#         dilation=(DH, DW),
#         padding=(PH, PW),
#         stride=(SH, SW)
#     )

# col2im

In [11]:
@triton.jit
def _col2im_kernel(
    dx_ptr,
    dcols_ptr,
    N, C, H, W,
    KH, KW, SH, SW, PH, PW, DH, DW,
    H_OUT, W_OUT, K_TOTAL,
    sxn, sxc, sxh, sxw,
    scn, sck, scl,
    BLOCK_X: tl.constexpr,
):
    pid = tl.program_id(0)
    L = H_OUT * W_OUT
    n = pid // L
    l = pid %  L

    oh = l // W_OUT
    ow = l %  W_OUT
    ih0 = oh * SH - PH
    iw0 = ow * SW - PW

    base_dx_n     = n * sxn
    base_dcols_nl = n * scn + l * scl

    k0 = 0
    while k0 < K_TOTAL:
        k      = k0 + tl.arange(0, BLOCK_X)
        k_mask = k < K_TOTAL

        r  = k %  (KH * KW)
        c  = k // (KH * KW)
        kh = r // KW
        kw = r %  KW

        ih = ih0 + kh * DH
        iw = iw0 + kw * DW

        inb = (ih >= 0) & (ih < H) & (iw >= 0) & (iw < W) & k_mask

        dval = tl.load(dcols_ptr + (base_dcols_nl + k * sck), mask=k_mask, other=0)
        tl.atomic_add(dx_ptr + (base_dx_n + c * sxc + ih * sxh + iw * sxw), dval, mask=inb)

        k0 += BLOCK_X

In [17]:
def col2im_triton(
    dcols: torch.Tensor,         # (N, K_TOTAL, L)
    out_shape,                   # (N, C, H, W)
    kernel_size, stride=1, padding=0, dilation=1,
    block_x: int = 256
):

    N, C, H, W = out_shape
    KH, KW = kernel_size
    SH, SW = stride
    PH, PW = padding
    DH, DW = dilation

    # Восстанавливаем H_OUT, W_OUT и L = H_OUT*W_OUT из размера dcols
    # dcols: (N, K_TOTAL, L), где K_TOTAL = C*KH*KW
    K_TOTAL = C * KH * KW
    assert dcols.shape[1] == K_TOTAL
    L = dcols.shape[2]

    # Если нужно: можно вычислить H_OUT,W_OUT через формулу, но часто у тебя уже есть (например из im2col).
    # Здесь восстановим через прямую формулу:
    H_OUT, W_OUT = _out_hw(H, W, KH, KW, SH, SW, PH, PW, DH, DW)
    assert H_OUT * W_OUT == L, "L не совпадает с H_OUT*W_OUT — проверь параметры."

    # Выходной градиент по входу (или просто собранное изображение) — аккумулируем в fp32
    dx = torch.zeros((N, C, H, W), device=dcols.device, dtype=torch.float32)

    sxn, sxc, sxh, sxw = dx.stride()
    scn, sck, scl = dcols.stride()

    grid = (N * L,)
    _col2im_kernel[grid](
        dx, dcols,
        N, C, H, W,
        KH, KW, SH, SW, PH, PW, DH, DW,
        H_OUT, W_OUT, K_TOTAL,
        sxn, sxc, sxh, sxw,
        scn, sck, scl,
        BLOCK_X=block_x,
        num_warps=4, num_stages=2,
    )
    return dx

In [18]:
N, C, H, W = 1, 3, 2000, 2000
KH, KW = 3, 3
SH, SW = 1, 1
PH, PW = 1, 1
DH, DW = 1, 1
block_k = 256
device = "cuda"

# ---- данные ----
x = torch.arange(N*C*H*W, dtype=torch.float32, device=device).reshape(N, C, H, W)


# ---- 1) эталонный unfold ----
cols_ref = F.unfold(
    x, kernel_size=(KH, KW),
    dilation=(DH, DW),
    padding=(PH, PW),
    stride=(SH, SW)
)  # (N, C*KH*KW, L)

# ---- 2) твой im2col_triton ----
cols_tr, (H_out, W_out) = im2col_triton(
    x,
    kernel_size=(KH, KW),
    stride=(SH, SW),
    padding=(PH, PW),
    dilation=(DH, DW),
    block_k=block_k
) 

x_sum_ref = F.fold(
    cols_ref, output_size=(H, W),
    kernel_size=(KH, KW),
    dilation=(DH, DW),
    padding=(PH, PW),
    stride=(SH, SW)
)  # (N, C, H, W)

# твой col2im (должен быть определён: col2im_triton)
x_sum_tr = col2im_triton(
    cols_tr, out_shape=(N, C, H, W),
    kernel_size=(KH, KW),
    stride=(SH, SW),
    padding=(PH, PW),
    dilation=(DH, DW),
    block_x=256
).to(x_sum_ref.dtype)
max_abs = (cols_tr - cols_ref).abs().max().item()
print(f"[im2col] max |diff| = {max_abs:.6g}")

[im2col] max |diff| = 0


# GEMM

In [ ]:
import torch
import triton
import triton.language as tl

@triton.autotune(
    configs=[
        triton.Config({"BLOCK_M": 128, "BLOCK_N": 64,  "BLOCK_K": 64}, num_warps=4, num_stages=2),
        triton.Config({"BLOCK_M": 64,  "BLOCK_N": 128, "BLOCK_K": 64}, num_warps=4, num_stages=2),
        triton.Config({"BLOCK_M": 128, "BLOCK_N": 128, "BLOCK_K": 32}, num_warps=8, num_stages=3),
    ],
    key=["M", "N", "K"],
)
@triton.jit
def _gemm_kernel(
    a_ptr, b_ptr, c_ptr,
    M, N, K,
    lda, ldb, ldc,                       # row-major: lda=K, ldb=N, ldc=N
    BLOCK_M: tl.constexpr,
    BLOCK_N: tl.constexpr,
    BLOCK_K: tl.constexpr,
    OUT_FP16: tl.constexpr,              # хранить C в fp16 или fp32
):
    pid_m = tl.program_id(0)
    pid_n = tl.program_id(1)

    offs_m = pid_m * BLOCK_M + tl.arange(0, BLOCK_M)   # [BM]
    offs_n = pid_n * BLOCK_N + tl.arange(0, BLOCK_N)   # [BN]
    mask_m = offs_m < M
    mask_n = offs_n < N

    acc = tl.zeros((BLOCK_M, BLOCK_N), dtype=tl.float32)

    k0 = 0
    while k0 < K:
        offs_k = k0 + tl.arange(0, BLOCK_K)            # [BK]
        mask_k = offs_k < K

        a = tl.load(a_ptr + (offs_m[:, None] * lda + offs_k[None, :]),
                    mask=mask_m[:, None] & mask_k[None, :], other=0)
        b = tl.load(b_ptr + (offs_k[:, None] * ldb + offs_n[None, :]),
                    mask=mask_k[:, None] & mask_n[None, :], other=0)

        acc += tl.dot(a, b, out_dtype=tl.float32)
        k0 += BLOCK_K

    c = acc.to(tl.float16) if OUT_FP16 else acc
    tl.store(c_ptr + (offs_m[:, None] * ldc + offs_n[None, :]),
             c, mask=mask_m[:, None] & mask_n[None, :])


def gemm_triton(A: torch.Tensor, B: torch.Tensor, out_dtype=torch.float16) -> torch.Tensor:
    """
    A: (M, K), B: (K, N)  -> C: (M, N)
    - Accumulation fp32, запись в fp16 или fp32.
    - Требует CUDA и contiguous row-major.
    """
    assert A.is_cuda and B.is_cuda
    assert A.dim() == 2 and B.dim() == 2
    M, K = A.shape
    Kb, N = B.shape
    assert K == Kb

    A = A.contiguous()
    B = B.contiguous()

    C = torch.empty((M, N), device=A.device,
                    dtype=(torch.float16 if out_dtype == torch.float16 else torch.float32))

    lda = A.stride(0)  # row-major: lda == K
    ldb = B.stride(0)  # row-major: ldb == N
    ldc = C.stride(0)  # row-major: ldc == N

    grid = (triton.cdiv(M, 128), triton.cdiv(N, 128))
    _gemm_kernel[grid](
        A, B, C,
        M, N, K,
        lda, ldb, ldc,
        OUT_FP16=(out_dtype == torch.float16),
    )
    return C